In [1]:
import psycopg2
# to install from commandline: conda install -c anaconda psycopg2
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)


In [2]:
#need to be connected to vpn
#server information is stored in config.py file 
import config 
conn = psycopg2.connect(dbname=config.dbname, user=config.user, host=config.host, port=config.port,password=config.password)
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciv" + ';'

From Garrett's notebooks: https://github.com/geickelb/mimiciii-antibiotics-opensource/blob/master/src/SQL/sofa/mort.sql

with co as (
select subject_id,hadm_id,icustay_id,
(date_part('day', dod::timestamp - intime::timestamp) + 1) 
as mortality 
from public.pan
)
select co.*, 
case when mortality <= 28 then 1 else 0 end as mort_28, /*variable name should not start with number*/
case when mortality <= 90 then 1 else 0 end as mort_90
from co where mortality is not null


From Github discussions: 
https://github.com/MIT-LCP/mimic-code/issues/1199
Mortality only really captured in hospital, not out of hospital, but that's okay for our purposes 



In [ ]:
select hadm_id, admittime, 	dischtime, deathtime, hospital_expire_flag, admission_location, discharge_location 
from `physionet-data.mimic_core.admissions`
limit 20

Outcomes of interest:
Mortality
Discharge disposition


In [ ]:
from mimiciv.

In [10]:
sql = query_schema + """
select *
from mimiciv.admissions
limit 5
"""
df = pd.read_sql_query(sql,conn) 
df


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,None,ELECTIVE,None,HOME,Other,ENGLISH,SINGLE,UNKNOWN,None,None,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,None,ELECTIVE,None,HOME,Other,ENGLISH,None,WHITE,None,None,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,None,ELECTIVE,None,HOME,Other,ENGLISH,None,UNKNOWN,None,None,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,None,ELECTIVE,None,HOME,Other,ENGLISH,None,OTHER,None,None,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,None,ELECTIVE,None,HOME,Other,ENGLISH,None,BLACK/AFRICAN AMERICAN,None,None,0


In [11]:
sql = query_schema + """
select hadm_id, admittime, 	dischtime, deathtime, hospital_expire_flag, admission_location, discharge_location 
from mimiciv.admissions
"""
admit = pd.read_sql_query(sql,conn) 
admit


,hadm_id,admittime,dischtime,deathtime,hospital_expire_flag,admission_location,discharge_location
0,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaT,0,None,HOME
1,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaT,0,None,HOME
2,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaT,0,None,HOME
3,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaT,0,None,HOME
4,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaT,0,None,HOME
...,...,...,...,...,...,...,...
523735,20786062,2180-09-17 00:00:00,2180-09-18 13:37:00,NaT,0,PHYSICIAN REFERRAL,HOME
523736,20943099,2147-08-01 17:41:00,2147-08-02 17:30:00,NaT,0,TRANSFER FROM HOSPITAL,HOME
523737,23176714,2189-03-01 00:58:00,2189-03-02 15:22:00,NaT,0,PHYSICIAN REFERRAL,HOME
523738,22347500,2138-05-31 00:00:00,2138-06-04 16:50:00,NaT,0,PHYSICIAN REFERRAL,HOME HEALTH CARE


In [15]:
admit.discharge_location.value_counts()

HOME                            222173
HOME HEALTH CARE                 86026
SKILLED NURSING FACILITY         46207
REHAB                            11328
DIED                              9238
CHRONIC/LONG TERM ACUTE CARE      7591
HOSPICE                           3679
ACUTE HOSPITAL                    3499
AGAINST ADVICE                    2718
PSYCH FACILITY                    2417
OTHER FACILITY                    1455
ASSISTED LIVING                    584
HEALTHCARE FACILITY                168
Name: discharge_location, dtype: int64

In [14]:
admit.admission_location.value_counts()

EMERGENCY ROOM                            245744
PHYSICIAN REFERRAL                        127494
TRANSFER FROM HOSPITAL                     39121
WALK-IN/SELF REFERRAL                      16660
CLINIC REFERRAL                            10670
PROCEDURE SITE                              8449
PACU                                        6067
INTERNAL TRANSFER TO OR FROM PSYCH          4467
TRANSFER FROM SKILLED NURSING FACILITY      4063
INFORMATION NOT AVAILABLE                    379
AMBULATORY SURGERY TRANSFER                  191
Name: admission_location, dtype: int64

In [16]:
admit.hospital_expire_flag.value_counts() #pretty close with discharge_location=died

0    514390
1      9350
Name: hospital_expire_flag, dtype: int64

In [ ]:
Let's clean up some of these names - 

Home = 
HOME                            222173
HOME HEALTH CARE                 86026
AGAINST ADVICE                    2718
PSYCH FACILITY                    2417

Rehab = 
REHAB                            11328

LTACH = 
ACUTE HOSPITAL                    3499
CHRONIC/LONG TERM ACUTE CARE      7591
ASSISTED LIVING                    584
HEALTHCARE FACILITY                168

SNF = 
OTHER FACILITY                    1455
SKILLED NURSING FACILITY         46207

Hospice = 
HOSPICE                           3679

Died = 
DIED                              9238


In [21]:
admit['discharge_location']=admit['discharge_location'].replace({
    'HOME': 'Home',
    'HOME HEALTH CARE': 'Home',
    'AGAINST ADVICE': 'Home',
    'PSYCH FACILITY': 'Home',
    'REHAB': 'Rehab',
    'ACUTE HOSPITAL': 'LTACH',
    'CHRONIC/LONG TERM ACUTE CARE': 'LTACH',
    'ASSISTED LIVING': 'LTACH',
    'HEALTHCARE FACILITY': 'LTACH',
    'SKILLED NURSING FACILITY': 'SNF',
    'OTHER FACILITY': 'SNF',
    'HOSPICE': 'Hospice',
    'DIED': 'Died'
})

In [22]:
admit.head()

,hadm_id,admittime,dischtime,deathtime,hospital_expire_flag,admission_location,discharge_location
0,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaT,0,None,Home
1,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaT,0,None,Home
2,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaT,0,None,Home
3,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaT,0,None,Home
4,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaT,0,None,Home


In [23]:
admit.discharge_location.value_counts()

Home       313334
SNF         47662
LTACH       11842
Rehab       11328
Died         9238
Hospice      3679
Name: discharge_location, dtype: int64

In [24]:
admit.to_csv("discharge_disposition_clean2-20-22.csv")